<a href="https://colab.research.google.com/github/fernandolievano/ProyectoFinal_PP1/blob/entrega-final%2Fparcial-2/ProyectoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # 1. Introducción

In [ ]:
# @title
from urllib.request import urlopen
from IPython.display import HTML

RAW_URL = "https://raw.githubusercontent.com/fernandolievano/ProyectoFinal_PP1/main/assets/doc.html"
with urlopen(RAW_URL) as f:
    html = f.read().decode("utf-8", errors="replace")
HTML(html)

# 2. Configuración del entorno

### 2.1 Instalación de Librerías

In [ ]:
%pip install pandas numpy plotly

### 2.2 Importación de Módulos

In [ ]:
import pandas as pd
import plotly.express as px

### 2.3 Carga del Dataset

In [ ]:
dataset_url = "https://raw.githubusercontent.com/fernandolievano/ProyectoFinal_PP1/main/data/stack-overflow-developer-survey-2025.csv.gz"
df = pd.read_csv(dataset_url, compression='gzip')

print("✅ DataSet cargado con éxto")

### 2.4 Funciones y Variables Utilitarias

In [ ]:
# Funciones Reutilizables
def contar_valores_explodidos(dataframe, columna, separador):
    """
    Cuenta las incidencias de valores en una columna que contiene strings
    separados por un delimitador.
    :param dataframe:
    :param columna:
    :param separador:
    :return:
        Una Serie de pandas
    """
    if columna not in dataframe.columns:
        raise ValueError(f"Columna {columna} no encontrada")

    return (dataframe[columna]
            .str.split(separador)
            .explode()  # cada elemento de la lista se vuelve una fila
            .value_counts())


# Variables Reutilizables
n_encuestados = len(df)

# 3. Exploración Inicial del Dataset

### 3.1 Estructura y Dimensiones

In [ ]:
print(f"Dimensiones del Dataset: {df.shape}")
print(f"Número de filas: {df.shape[0]}")
print(f"Número de columnas: {df.shape[1]}")

### 3.2 Información general del Dataset

In [ ]:
df.info()

### 3.3 Head y Tail para muestras

In [ ]:
df.head()

In [ ]:
df.tail()

### 3.4 Estadísticas descriptivas

In [ ]:
df.describe().T

# 4. Análisis Exploratorio de Datos (EDA)
Llevaremos a cabo un análisis exploratorio de datos utilizando visualizaciones interactivas y estadísticas descriptivas, con el objetivo de comprender mejor los resultados de la encuesta hecha por Stack Overflow en 2025. Mediante este proceso podremos responder preguntas sobre lenguajes de programación más usados, tecnologías en tendencias, roles comunes y niveles salariales.

### 4.1 Verificación de nulos

In [ ]:
pd.set_option('display.max_rows', None)  # Activo display.max_rows para poder ver todos mis columnas nulas

tiene_nulos = df.isnull().sum() > 0  # Máscara para mostrar solo resultados con valores nulos
nulos = df.isnull().sum()[tiene_nulos]
nulos_cantidad = len(nulos)
nulos_porcentaje = (nulos / n_encuestados) * 100

print(f"❗ Encontramos {nulos_cantidad} columnas con datos nulos")

nulos_resumen = pd.DataFrame({
    'Tipo de Dato': df.dtypes[tiene_nulos],
    'Valores Nulos': nulos,
    '% Nulos': nulos_porcentaje
})

nulos_resumen

In [ ]:
pd.reset_option('display.max_rows', None)  # Desactivo de nuevo display.max_rows

### 4.2 Variables con Alto Porcentaje de Nulos

En este gráfico se muestran las variables con mayor porcentaje de valores faltantes.
Dado que superan ampliamente el 80% de nulos, pueden considerarse prescindibles para el análisis, ya que su nivel de completitud es insuficiente para obtener insights confiables.

In [ ]:
sobre_80_porciento = nulos_porcentaje > 80  # Máscara obtener resultados mayores a 80%
nulos_altos = nulos_porcentaje[sobre_80_porciento].sort_values(ascending=False)

# Gráfico
fig_altos_nulos = px.bar(
    nulos_altos,
    color='value',
    orientation='h',
    title='⚠️ Variables con más del 80% de valores nulos',
    labels={'value': 'Porcentaje de Nulos', 'index': 'Variable'},
    color_continuous_scale='Reds'
)
fig_altos_nulos.update_layout(
    height=40 * len(nulos_altos),
    font=dict(
        size=14,
        color='#fff',
    ),
    title=dict(
        xanchor='center',
        x=0.5
    ),
    paper_bgcolor='black',
    plot_bgcolor='black',
)
fig_altos_nulos.show()

### 4.3 Variables Clave y su Completitud
Se muestran variables fundamentales del análisis (como país, educación y años de experiencia) que presentan un bajo porcentaje de valores faltantes, lo que permite tratarlos sin afectar la calidad de los resultados.

In [ ]:
variables_clave = ['Country', 'Employment', 'EdLevel', 'WorkExp', 'YearsCode', 'ConvertedCompYearly']
nulos_claves = nulos[variables_clave].sort_values(ascending=False)

# Gráfico
fig_nulos_clave = px.bar(
    nulos_claves,
    title='🔑 Cantidad de nulos en variables clave',
    labels={'value': 'Cantidad de Nulos', 'index': 'Variable'},
    color='value',
    color_continuous_scale='sunsetdark'
)
fig_nulos_clave.update_layout(
    font=dict(
        size=14,
        color='#fff',
    ),
    title=dict(
        xanchor='center',
        x=0.5
    ),
    paper_bgcolor='black',
    plot_bgcolor='black',
)
fig_nulos_clave.show()

### 4.4 Estadísticas Descriptivas

4.4.1 Resúmenes numéricos de variables continuas

In [ ]:
print("📊 ESTADÍSTICAS DESCRIPTIVAS - VARIABLES NUMÉRICAS")
variables_numericas = ['ConvertedCompYearly', 'WorkExp', 'YearsCode']
df_numericas = df[variables_numericas].dropna()

print(df_numericas.describe())

4.4.2 Análisis de Correlacionales

In [ ]:
matriz_de_correlacionales = df_numericas.corr()

fig_corr = px.imshow(
    matriz_de_correlacionales,
    title='Matriz de Correlación entre Variables Numéricas',
    color_continuous_scale='RdBu_r',
    aspect='auto'
)
fig_corr.update_layout(
    paper_bgcolor='black',
    font=dict(color='white', size=14)
)
fig_corr.show()

 # Análisis de hipótesis
 En esta sección se abordan las preguntas planteadas al inicio del proyecto, utilizando gráficos interactivos y resúmenes numéricos para explorarlas en detalle.

## 1. ¿Qué lenguajes de programación están en auge?

In [ ]:
# Usamos la función "contar_valores_explodidos" para dividir los valores
# de la columna 'LanguageHaveWorkedWith' (separados por ';') y obtener
# un conteo de los lenguajes más utilizados.
lenguajes_usados_exploded = contar_valores_explodidos(dataframe=df, columna='LanguageHaveWorkedWith', separador=';')
lenguajes_usados_exploded = lenguajes_usados_exploded.sort_values(ascending=True)

fig_lenguajes_usados = px.bar(
    lenguajes_usados_exploded,
    title='👨‍💻 Lenguajes de programación más utilizados en 2025',
    color='value',
    color_continuous_scale='pinkyl',
    orientation='h',
    labels={'value': 'Uso', 'LanguageHaveWorkedWith': 'Lenguaje de Programación'},
)
fig_lenguajes_usados.update_layout(
    font=dict(
        size=14,
        color='#fff',
    ),
    paper_bgcolor='black',
    plot_bgcolor='black',
    height=36 * len(lenguajes_usados_exploded),
    title=dict(
        xanchor='center',
        x=0.5
    ),
)

fig_lenguajes_usados.show()

> 🔍 **El ecosistema web domina el uso**. JavaScript, HTML/CSS y SQL lideran con Python muy cerca. Además, TypeScript se posiciona por encima de Java/C#, evidenciando la consolidación del front-end “profesional”.

In [ ]:
# === Expectativas de Lenguajes (auge) ===
# Gap (pp) = % que desea usar − % que usa hoy, ambos sobre el total de encuestados.

# 1) Conteos de "deseo"
lenguajes_deseados = contar_valores_explodidos(
    dataframe=df,
    columna='LanguageWantToWorkWith',
    separador=';'
)

# 2) Normalización a % del total de encuestados
lenguajes_deseados_pct = (lenguajes_deseados / n_encuestados) * 100
lenguajes_usados_pct   = (lenguajes_usados_exploded / n_encuestados) * 100

# 3) Alineación por índice (si un lenguaje aparece en una lista y no en la otra, tomar 0)
todos = lenguajes_deseados_pct.index.union(lenguajes_usados_pct.index)
deseo_pp = lenguajes_deseados_pct.reindex(todos, fill_value=0.0)
uso_pp   = lenguajes_usados_pct.reindex(todos, fill_value=0.0)

# 4) Gap de auge (pp) y filtro de positivos
lenguajes_auge = (deseo_pp - uso_pp)
lenguajes_auge = lenguajes_auge[lenguajes_auge > 0].sort_values(ascending=True)

# 5) Gráfico
import plotly.express as px

fig_lenguajes_auge = px.bar(
    lenguajes_auge,
    title="📈 Lenguajes de Programación en Auge en 2025",
    color="value",
    color_continuous_scale="tempo",
    orientation="h",
    labels={"index": "Lenguajes de Programación", "value": "% auge (pp)"},
)

# Escala de color consistente (desde 0 hasta el máximo) y título del colorbar
fig_lenguajes_auge.update_coloraxes(
    cmin=0,
    cmax=float(lenguajes_auge.max()),
    colorbar_title="% auge (pp)",
)

# Hover claro
fig_lenguajes_auge.update_traces(
    hovertemplate="Lenguaje: %{y}<br>Auge: %{x:.2f} pp<extra></extra>"
)

# Estilo general y altura con tope
fig_lenguajes_auge.update_layout(
    font=dict(size=14, color="#fff"),
    paper_bgcolor="black",
    plot_bgcolor="black",
    margin=dict(l=220, r=40, t=80, b=40),
    height=min(36 * len(lenguajes_auge), 1200),
    title=dict(
        xanchor="center",
        x=0.5,
        text=(
            "📈 Lenguajes de Programación en Auge en 2025"
            f"<br><sup>Gap = deseo − uso (pp) sobre el total de encuestados, n = {n_encuestados}</sup>"
        ),
    ),
)

fig_lenguajes_auge.show()


> 🧠 El gráfico muestra los lenguajes con gap positivo (deseo − uso, en puntos porcentuales sobre el total de encuestados), es decir, aquellos que hoy son más deseados que utilizados. Este patrón sugiere potencial de adopción en el corto/mediano plazo: lenguajes emergentes tienden a liderar por interés futuro elevado pese a una base actual pequeña. En términos prácticos, estos resultados apuntan a oportunidades de formación y a una posible reorientación de stack hacia tecnologías con mayor tracción aspiracional.

***Nota***: la lectura debe considerar el tamaño de muestra por lenguaje (los de base reducida son más volátiles) y que este indicador refleja intención declarada, no adopción efectiva.

## 2. ¿Qué tecnologías muestran menor interés futuro?

In [ ]:
# === Tecnologías que pierden interés (gap negativo) ===
# Gap (pp) = % que desea usar − % que usa hoy, ambos sobre el total de encuestados.
lenguajes_retroceso = (lenguajes_deseados_pct - lenguajes_usados_pct)

# Filtramos sólo retroceso (< 0) y ordenamos de más negativo (mayor retroceso) a menos negativo.
lenguajes_retroceso = lenguajes_retroceso[lenguajes_retroceso < 0].sort_values(ascending=True)

# Gráfico de barras horizontal (Series -> usa 'index' y 'value')
import plotly.express as px

fig_lenguajes_retroceso = px.bar(
    lenguajes_retroceso,
    title="📉 Tecnologías que pierden interés en 2025",
    color='value',                       # escala por magnitud del retroceso
    color_continuous_scale='pinkyl_r',
    orientation='h',
    labels={'index': 'Lenguajes de Programación', 'value': '% retroceso (pp)'}
)

# Escala de color consistente (máximo en 0) y título del colorbar
fig_lenguajes_retroceso.update_coloraxes(
    cmin=lenguajes_retroceso.min(),
    cmax=0,
    colorbar_title="% retroceso (pp)"
)

# Hover claro
fig_lenguajes_retroceso.update_traces(
    hovertemplate="Lenguaje: %{y}<br>Retroceso: %{x:.2f} pp<extra></extra>"
)

# Estilo general
fig_lenguajes_retroceso.update_layout(
    font=dict(size=14, color="#fff"),
    paper_bgcolor="black",
    plot_bgcolor="black",
    margin=dict(l=220, r=40, t=80, b=40),
    height=min(36 * len(lenguajes_retroceso), 1200),
    title=dict(
        xanchor="center",
        x=0.5,
        text=f"📉 Tecnologías que pierden interés en 2025"
             f"<br><sup>Gap = deseo − uso (pp) sobre el total de encuestados, n = {n_encuestados}</sup>"
    ),
)

fig_lenguajes_retroceso.show()


> 🧠 El gráfico presenta las tecnologías con gap negativo. La diferencia entre quienes desean usarlas y quienes las usan hoy, en puntos porcentuales sobre el total de encuestados, lo que indica desinterés relativo o pérdida de relevancia a futuro. En varias tecnologías masivas (p. ej., JS/HTML-CSS/SQL) el retroceso sugiere saturación/estabilidad más que abandono: mantienen alta adopción, pero una proporción menor planea seguir eligiéndolas. Estos resultados invitan a diversificar habilidades hacia opciones con mayor tracción aspiracional, sin sobrerreaccionar: la lectura debe considerar el tamaño de muestra por lenguaje y que la métrica refleja intención declarada, no adopción efectiva.